In [1]:
!pip install selenium

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\sq\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

categories = {
    "smartphones": "https://www.banggood.com/Wholesale-Smartphones-c-1861.html",
    "laptops": "https://www.banggood.com/Wholesale-Laptops-c-2564.html",
    "watches": "https://www.banggood.com/Wholesale-Smart-Watches-c-4362.html",
    "rc_toys": "https://www.banggood.com/Wholesale-RC-Toys-c-133.html",
    "headphones": "https://www.banggood.com/Wholesale-Headphones-c-3222.html"
}

all_products = []

def scrape_page(url, category):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.select(".goodlist_1 .good-item")

    for item in items:
        name = item.select_one(".title").get_text(strip=True) if item.select_one(".title") else None
        price = item.select_one(".price").get_text(strip=True) if item.select_one(".price") else None
        rating = item.select_one(".star").get_text(strip=True) if item.select_one(".star") else None
        reviews = item.select_one(".review").get_text(strip=True) if item.select_one(".review") else None
        url = item.select_one("a")["href"] if item.select_one("a") else None

        all_products.append({
            "category": category,
            "name": name,
            "price": price,
            "rating": rating,
            "reviews": reviews,
            "url": url
        })


# PAGINATION (first 5 pages)
for category_name, category_url in categories.items():
    print(f"Scraping: {category_name}")
    
    for page in range(1, 6):
        paginated_url = f"{category_url}?page={page}"
        scrape_page(paginated_url, category_name)
        time.sleep(1)

# Save to CSV
df = pd.DataFrame(all_products)
df.to_csv("banggood_products_bs.csv", index=False)

print("Done! Total products scraped:", len(df))



Scraping: smartphones


TooManyRedirects: Exceeded 30 redirects.

In [3]:
!pip install webdriver-manager pandas


   ---------------------------------------- 0/2 [python-dotenv]
   -------------------- ------------------- 1/2 [webdriver-manager]
   -------------------- ------------------- 1/2 [webdriver-manager]
   -------------------- ------------------- 1/2 [webdriver-manager]
   ---------------------------------------- 2/2 [webdriver-manager]




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\sq\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

categories = {
    "smartphones": "https://www.banggood.com/Wholesale-Smartphones-c-1861.html",
    "laptops": "https://www.banggood.com/Wholesale-Laptops-c-2564.html",
    "watches": "https://www.banggood.com/Wholesale-Smart-Watches-c-4362.html",
    "rc_toys": "https://www.banggood.com/Wholesale-RC-Toys-c-133.html",
    "headphones": "https://www.banggood.com/Wholesale-Headphones-c-3222.html"
}

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")
options.add_argument("--ignore-certificate-errors")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

all_products = []

def scrape_category(url, category):
    for page in range(1, 6):
        try:
            paginated_url = f"{url}?page={page}"
            print("Scraping:", paginated_url)
            driver.get(paginated_url)
            time.sleep(3)

            products = driver.find_elements(By.CSS_SELECTOR, ".good-item")

            if not products:
                print("No products found on this page!")
                continue

            for p in products:
                name = p.find_element(By.CSS_SELECTOR, ".title").text if p.find_elements(By.CSS_SELECTOR, ".title") else None
                price = p.find_element(By.CSS_SELECTOR, ".price").text if p.find_elements(By.CSS_SELECTOR, ".price") else None
                rating = p.find_element(By.CSS_SELECTOR, ".star").text if p.find_elements(By.CSS_SELECTOR, ".star") else None
                reviews = p.find_element(By.CSS_SELECTOR, ".review").text if p.find_elements(By.CSS_SELECTOR, ".review") else None
                product_url = p.find_element(By.CSS_SELECTOR, "a").get_attribute("href") if p.find_elements(By.CSS_SELECTOR, "a") else None

                all_products.append({
                    "category": category,
                    "name": name,
                    "price": price,
                    "rating": rating,
                    "reviews": reviews,
                    "url": product_url
                })

        except Exception as e:
            print("Error occurred:", e)


for cat_name, cat_url in categories.items():
    print(f"--- Scraping category: {cat_name} ---")
    scrape_category(cat_url, cat_name)

driver.quit()

df = pd.DataFrame(all_products)
df.to_csv("banggood_products.csv", index=False)

print("Done! Total products:", len(df))


In [7]:
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import os

# --- STEP 0: Log Function ---
def log_progress(message):
    timestamp_format = '%Y-%m-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    print(f"{timestamp} : {message}")
    try:
        with open('code_log.txt', 'a', encoding='utf-8') as f:
            f.write(timestamp + ' : ' + message + '\n')
    except:
        pass

# --- STEP 1: Extract Function ---
def extract(url, category_name):
    log_progress(f"Starting extraction for Category: {category_name}")
    
    # NOTE: You may need to configure your WebDriver path or use a WebDriver Manager
    # options = webdriver.ChromeOptions()
    # options.add_argument("--start-maximized")
    # driver = webdriver.Chrome(options=options)
    
    # Using headless mode is often better for scraping (uncomment if needed)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(options=options)
    
    try:
        driver.get(url)
        log_progress(f"URL Opened: {url}")
        
        # Scrolling
        log_progress("Scrolling to load products...")
        for i in range(1, 6):
            driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight * {i/6});")
            time.sleep(2)
        time.sleep(3)
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        data = []
        
        # Finding Cards
        cards = soup.select(".product-item")
        if not cards:
            cards = soup.select(".p-wrap") # Fallback
            
        log_progress(f"Found {len(cards)} cards in {category_name}.")

        for c in cards:
            try:
                # Name
                link_tag = c.select_one("a")
                name = link_tag.get("title") if link_tag else None
                if not name: 
                    img_tag = c.select_one("img")
                    name = img_tag.get("alt") if img_tag else None

                # Price
                price_tag = c.select_one(".price")
                if not price_tag: price_tag = c.select_one(".price-box")
                price = price_tag.get_text(strip=True) if price_tag else "N/A"
                
                # URL
                product_url = link_tag["href"] if link_tag else "N/A"
                if product_url != "N/A" and not product_url.startswith("http"):
                    product_url = "https://www.banggood.com" + product_url

                # Add Data if Name exists
                if name:
                    data.append({
                        "Category": category_name,
                        "Name": name,
                        "Price": price,
                        "Rating": "N/A", # Placeholder
                        "Reviews": "0",  # Placeholder
                        "URL": product_url
                    })
            except:
                continue

        df = pd.DataFrame(data)
        log_progress(f"Finished {category_name}. Extracted: {len(df)} rows")
        return df
        
    except Exception as e:
        log_progress(f"Error in {category_name}: {e}")
        return pd.DataFrame()
    finally:
        driver.quit()

# --- MAIN EXECUTION BLOCK (Loop for NEW Categories) ---
if __name__ == "__main__":
    
    # 5 NEW Categories and URLs for Banggood
    categories = {
        "Home Garden": "https://www.banggood.com/Wholesale-Home-and-Garden-ca-11001.html",
        "Jewelry": "https://www.banggood.com/Wholesale-Jewelry-and-Watch-ca-16001.html",
        "Computers": "https://www.banggood.com/Wholesale-Computer-and-Office-ca-15001.html",
        "Security": "https://www.banggood.com/Wholesale-Security-System-ca-18001.html",
        "Apparel": "https://www.banggood.com/Wholesale-Apparel-and-Accessories-ca-17001.html"
    }

    NEW_CSV_FILENAME = "my_new_scrape_data.csv" # <-- NEW CSV FILE NAME

    all_data_frames = [] # Yahan sab jama karenge

    print("--- BATCH EXTRACTION STARTED ---")

    # Loop chalao har category ke liye
    for cat_name, cat_url in categories.items():
        print(f"\nProcessing: **{cat_name}**...")
        df_temp = extract(cat_url, cat_name)
        
        if not df_temp.empty:
            all_data_frames.append(df_temp)
        else:
            print(f"Skipping {cat_name} due to error.")

    # Sab ko Join (Concat) karo
    if all_data_frames:
        final_df = pd.concat(all_data_frames, ignore_index=True)
        
        print("\n--- FINAL SUCCESS ---")
        print(f"Total Products Scraped: {len(final_df)}")
        print(final_df['Category'].value_counts()) # Har category me kitne items aye
        
        # Final Save
        final_df.to_csv(NEW_CSV_FILENAME, index=False)
        print(f"Data saved to **'{NEW_CSV_FILENAME}'**")
    else:
        print("No data extracted from any category.")

--- BATCH EXTRACTION STARTED ---

Processing: **Home Garden**...
2025-11-23-03:24:59 : Starting extraction for Category: Home Garden
2025-11-23-03:25:42 : URL Opened: https://www.banggood.com/Wholesale-Home-and-Garden-ca-11001.html
2025-11-23-03:25:42 : Scrolling to load products...
2025-11-23-03:25:56 : Found 60 cards in Home Garden.
2025-11-23-03:25:56 : Finished Home Garden. Extracted: 30 rows

Processing: **Jewelry**...
2025-11-23-03:25:58 : Starting extraction for Category: Jewelry
2025-11-23-03:26:22 : URL Opened: https://www.banggood.com/Wholesale-Jewelry-and-Watch-ca-16001.html
2025-11-23-03:26:22 : Scrolling to load products...
2025-11-23-03:26:35 : Found 60 cards in Jewelry.
2025-11-23-03:26:35 : Finished Jewelry. Extracted: 30 rows

Processing: **Computers**...
2025-11-23-03:26:37 : Starting extraction for Category: Computers
2025-11-23-03:27:21 : URL Opened: https://www.banggood.com/Wholesale-Computer-and-Office-ca-15001.html
2025-11-23-03:27:21 : Scrolling to load products